In [ ]:
from langchain.prompts import ChatPromptTemplate 
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import YoutubeLoader
from operator import itemgetter
import markdown
import datetime
import pytube

# from services.email.email_sender import send_email
# from services.database import users_collection
# from models.read_later import ReadLater
from fastapi import HTTPException
from dotenv import load_dotenv
import os
load_dotenv()


def chain_summarization(content: str, size: str = 'medium'):
    
    if size == 'small':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar textos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de uma página web e gerar uma sumarização curta que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}')] 
    elif size == 'medium':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar textos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de uma página web e gerar uma sumarização de tamanho médio que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}')]
    elif size == 'large':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar textos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de uma página web e gerar uma sumarização longa e detalhada que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}.')]

    prompt = ChatPromptTemplate.from_messages(messages)
    llm = ChatOpenAI(model = 'gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
    chain = (
        {
            'texto': itemgetter('texto')
        }
        | prompt
        | llm
        | StrOutputParser()
    )

    resume = chain.invoke({'texto': content.content})
    return resume

def youtube_summarization(url: str, size: str = 'medium'):
    
    url = url.strip()
    loader = YoutubeLoader.from_youtube_url(url, language=['pt', 'en'], translation='pt')
    texto = loader.load()
    pytube_loader = pytube.YouTube(url)
    title = pytube_loader.title
    
    if size == 'small':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar vídeos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de um vídeo e gerar uma sumarização curta que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}')] 
    elif size == 'medium':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar vídeos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de um vídeo e gerar uma sumarização de tamanho médio que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}')]
    elif size == 'large':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar vídeos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de um vídeo e gerar uma sumarização longa e detalhada que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}.')]

    prompt = ChatPromptTemplate.from_messages(messages)
    llm = ChatOpenAI(model = 'gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
    chain = (
        {
            'texto': itemgetter('texto')
        }
        |prompt
        | llm
        |StrOutputParser()
    )

    resume = chain.invoke({'texto': texto})
    return resume, title


In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

page_url = "https://python.langchain.com/docs/how_to/chatbots_memory/"

loader = WebBaseLoader(web_paths=[page_url])
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs) == 1
doc = docs[0]